In [1]:
from googleapiclient.discovery import build
import csv
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import scipy
import networkx as nx
from itertools import combinations
import math

In [2]:
DEVELOPER_KEY = open("../esercizi_classe/api_key.txt").read()
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

Ricerca e salvataggio di tutti i video per canale nel periodo temporale

In [3]:
channels = ["Romeo Agresti", "Il BiancoNero", "Colpo Gobbo","Luca Toselli","lAngolodiKinoshi"]

In [ ]:
def getIDfromName(name):
    request = youtube.search().list(
        part="snippet",
        q=name,
        type="channel",
        maxResults=5
    )
    response = request.execute()
    return response['items'][0]['id']['channelId']

def getChannelPlaylist(channel_id):
    request = youtube.channels().list(
        part="snippet,contentDetails",
        id=channel_id
    )
    response = request.execute()
    return response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

def get_videos_from_channel(playlist_id,channel_name):
    begin_date = datetime(2024,7,7)
    end_date = datetime(2025,3,24)
    video_ids_and_dates = []
    next_page_token = None
    while True:
        request = youtube.playlistItems().list(
            part="snippet",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()
        for item in response['items']:
            video_date = datetime.strptime(item['snippet']['publishedAt'], "%Y-%m-%dT%H:%M:%SZ")
            if begin_date <= video_date <= end_date:
                video_ids_and_dates.append((item['snippet']['resourceId']['videoId'], video_date))
        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
    with open(f"video_ids_{channel_name}.csv", "w", newline='') as csvfile:
        writer = csv.writer(csvfile)
        for video_id in video_ids_and_dates:
            writer.writerow([video_id[0],video_id[1].strftime("%Y-%m-%d")])
    print(f"Video IDs for {channel_name} saved to video_ids_{channel_name}.csv")

def get_csv_files(channels):
    for channel in channels:
        channel_id = getIDfromName(channel)
        playlist_id = getChannelPlaylist(channel_id)
        get_videos_from_channel(playlist_id,channel)
    

In [ ]:
get_csv_files(channels)

In [4]:
def get_video_views(video_id):
    """Get the view count for a specific video"""
    try:
        request = youtube.videos().list(
            part="statistics",
            id=video_id
        )
        response = request.execute()
        if response['items']:
            return int(response['items'][0]['statistics']['viewCount'])
        else:
            return 0
    except:
        return 0

def add_views_to_csv(channels):
    """Read existing CSV files and add view counts"""
    for channel in channels:
        # Read existing data
        video_data = []
        with open(f"video_ids_{channel}.csv", "r") as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                video_id = row[0]
                date = row[1]
                views = get_video_views(video_id)
                video_data.append([video_id, date, views])
                print(f"Channel: {channel}, Video: {video_id}, Views: {views}")
        
        # Write updated data with views
        with open(f"video_ids_{channel}_with_views.csv", "w", newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["Video ID", "Date", "Views"])  # Header
            for row in video_data:
                writer.writerow(row)
        
        print(f"Updated video data for {channel} saved to video_ids_{channel}_with_views.csv")

# Add views to all channel CSV files
add_views_to_csv(channels)

Channel: Romeo Agresti, Video: spe1z4ebmsw, Views: 28616
Channel: Romeo Agresti, Video: rznXkLkzLC4, Views: 24634
Channel: Romeo Agresti, Video: HfymSU0FvpU, Views: 31089
Channel: Romeo Agresti, Video: 63w9vrAHB7g, Views: 14410
Channel: Romeo Agresti, Video: hbP1fSUfYhg, Views: 14169
Channel: Romeo Agresti, Video: FaILsGTzUA8, Views: 14327
Channel: Romeo Agresti, Video: I_OiC7MLoN8, Views: 13898
Channel: Romeo Agresti, Video: vuWnmnM4xrQ, Views: 11977
Channel: Romeo Agresti, Video: gVhADjwA5Kc, Views: 19836
Channel: Romeo Agresti, Video: 5KXECIcHj4Q, Views: 9288
Channel: Romeo Agresti, Video: hLLa2lHo-6w, Views: 21008
Channel: Romeo Agresti, Video: PGdx1nufqEI, Views: 20240
Channel: Romeo Agresti, Video: ZioIZk31T44, Views: 14248
Channel: Romeo Agresti, Video: ZKXsnf3aGss, Views: 14536
Channel: Romeo Agresti, Video: C-LPz1x3uAs, Views: 21824
Channel: Romeo Agresti, Video: LiH7VzYhKRo, Views: 30578
Channel: Romeo Agresti, Video: pCV-lN81aso, Views: 9087
Channel: Romeo Agresti, Video: 1W

<p>Reperimento dei commenti:
<p>-per ogni commento salvo id, video commentato, autore, contenuto, data, likes, id commento a cui risponde (se c'è)

<img src="../esercizi_classe/Grafo_Canali.png" width="500">

In [30]:
class Comment:
    def __init__(self, id, video_id, content, author,date, likes, reply_to_id=None):
        self.id = id
        self.video_id = video_id
        self.content = content
        self.author = author
        self.date = date
        self.likes = likes
        self.reply_to_id = reply_to_id

In [ ]:
def get_comments_one_vid(video_id):
    comments = []
    request = youtube.commentThreads().list(
        part="snippet,replies",
        videoId=video_id,
        textFormat="plainText",
        maxResults=100
    )
    response = request.execute()
    comments.extend(get_comments_from_response(response["items"]))
    next_page_token = response.get("nextPageToken", None)
    while next_page_token:
        request = youtube.commentThreads().list(
            part="snippet,replies",
            videoId=video_id,
            textFormat="plainText",
            maxResults=100,
            pageToken=next_page_token
        )
        response = request.execute()
        comments.extend(get_comments_from_response(response["items"]))
        next_page_token = response.get("nextPageToken", None)
    return comments

def get_comments_from_response(items):
    comments = []
    for item in items:
        main_comment = item["snippet"]["topLevelComment"]
        comment_id = main_comment["id"]
        video_id = item["snippet"]["videoId"]
        author = main_comment["snippet"]["authorDisplayName"]
        content = main_comment["snippet"]["textDisplay"]
        date = datetime.strptime(main_comment["snippet"]["publishedAt"], "%Y-%m-%dT%H:%M:%SZ")
        likes = main_comment["snippet"]["likeCount"]
        comments.append(Comment(comment_id, video_id, content, author, date, likes))
        if "replies" in item:
            for reply in item["replies"]["comments"]:
                reply_id = reply["id"]
                reply_content = reply["snippet"]["textDisplay"]
                reply_author = reply["snippet"]["authorDisplayName"]
                reply_date = datetime.strptime(reply["snippet"]["publishedAt"], "%Y-%m-%dT%H:%M:%SZ")
                reply_likes = reply["snippet"]["likeCount"]
                comments.append(Comment(reply_id, video_id, reply_content, reply_author, reply_date, reply_likes, comment_id))
    return comments

def save_comments_csv(comments, channel_name):
    with open(f"comments_{channel_name}.csv", "w", newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Comment ID", "Video ID", "Content", "Author", "Date", "Likes", "Reply To ID"])
        for comment in comments:
            writer.writerow([comment.id, comment.video_id, comment.content, comment.author, comment.date.strftime("%Y-%m-%d"), comment.likes, comment.reply_to_id])
    print(f"Comments for {channel_name} saved to comments_{channel_name}.csv")

def get_comments_from_csv_file(channel):
    with open(f"video_ids_{channel}.csv", "r") as csvfile:
        reader = csv.reader(csvfile)
        video_ids = [row[0] for row in reader]
    all_comments = []
    for video_id in video_ids:
        comments = get_comments_one_vid(video_id)
        all_comments.extend(comments)
    save_comments_csv(all_comments, channel)


In [ ]:
for channel in channels:
    get_comments_from_csv_file(channel)

In [31]:
# Strutture ausiliarie:
# video_id -> canale
# video_id -> commentatori
# canale -> video_id
# canale -> commentatori
from collections import defaultdict

def video_id_to_channel(channels):
    video_id_to_channel = defaultdict(str)
    for channel in channels:
        with open(f"video_ids_{channel}.csv", "r") as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                video_id_to_channel[row[0]] = channel
    return video_id_to_channel

def channel_to_video_id(channels):
    channel_to_video_id = defaultdict(str)
    for channel in channels:
        with open(f"video_ids_{channel}.csv", "r") as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                if channel not in channel_to_video_id:
                    channel_to_video_id[channel] = []
                channel_to_video_id[channel].append(row[0])
    return channel_to_video_id

def video_id_to_commenters(channels):
    video_id_to_commenters = defaultdict(str)
    for channel in channels:
        with open(f"comments_{channel}.csv", "r", encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                video_id = row["Video ID"]
                author = row["Author"]
                if video_id not in video_id_to_commenters:
                    video_id_to_commenters[video_id] = set()
                video_id_to_commenters[video_id].add(author)
    return video_id_to_commenters

def channel_to_commenters(channels):
    channel_to_commenters = defaultdict(str)
    for channel in channels:
        with open(f"comments_{channel}.csv", "r", encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if channel not in channel_to_commenters:
                    channel_to_commenters[channel] = set()
                channel_to_commenters[channel].add(row["Author"])
    return channel_to_commenters

def user_to_video_id(channels):
    user_to_video_id = defaultdict(str)
    for channel in channels:
        with open(f"comments_{channel}.csv", "r", encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                author = row["Author"]
                video_id = row["Video ID"]
                if author not in user_to_video_id:
                    user_to_video_id[author] = set()
                user_to_video_id[author].add(video_id)
    return user_to_video_id

def channel_to_comments(channels):
    channel_to_comments = defaultdict(str)
    for channel in channels:
        with open(f"comments_{channel}.csv", "r", encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                Comment(row["Comment ID"], row["Video ID"], row["Content"], row["Author"], row["Date"], row["Likes"], row["Reply To ID"])
                if channel not in channel_to_comments:
                    channel_to_comments[channel] = []
                channel_to_comments[channel].append(Comment(row["Comment ID"], row["Video ID"], row["Content"], row["Author"], row["Date"], row["Likes"], row["Reply To ID"]))
    return channel_to_comments

def video_to_date(channels):
    video_to_date = defaultdict(str)
    for channel in channels:
        with open(f"video_ids_{channel}.csv", "r") as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                video_to_date[row[0]] = row[1]
    return video_to_date

def commenter_to_channels(channels):
    commenter_to_channels = defaultdict(str)
    for channel in channels:
        with open(f"comments_{channel}.csv", "r", encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                author = row["Author"]
                if author not in commenter_to_channels:
                    commenter_to_channels[author] = set()
                commenter_to_channels[author].add(channel)
    return commenter_to_channels


Channel_of = video_id_to_channel(channels)
Videos_of = channel_to_video_id(channels)
Commenters_of_video = video_id_to_commenters(channels)
Commenters_of_channel = channel_to_commenters(channels)
Videos_commented_by = user_to_video_id(channels)
Channels_commented_by = commenter_to_channels(channels)
Comments_of_channel = channel_to_comments(channels)
Date_of_video = video_to_date(channels)

In [ ]:
import json
print(json.dumps(Videos_of["Romeo Agresti"], indent=4))
print(Commenters_of_video["FaILsGTzUA8"])

In [ ]:
Comments_of_channel["Romeo Agresti"][0].likes

In [ ]:
class NodeCentralities:
    def __init__(self, node, degree, eigenvector_centrality, betweenness_centrality,degree_centrality):
        self.node = node
        self.degree = degree
        self.eigenvector_centrality = eigenvector_centrality
        self.betweenness_centrality = betweenness_centrality
        self.degree_centrality = degree_centrality

In [32]:
total_videos = []
for channel in channels:
    i = 0
    for video_id in Videos_of[channel]:
        total_videos.append(video_id)
        i += 1        

50 video per canale. Cocommentatori Comuni

In [33]:
def create_tuples(commenters1, commenters2,skip_self=True):
    tuples = []
    print("Inizio")
    for c1 in commenters1:
        for c2 in commenters2:
            if c1 == c2 and skip_self:
                continue
            tuples.append((c1, c2))
    return tuples

In [35]:
G = nx.Graph()

for video1, video2 in combinations(total_videos, 2):
    commenters1 = set(Commenters_of_video[video1])
    commenters2 = set(Commenters_of_video[video2])
    common_commenters = commenters1.intersection(commenters2)
    if len(common_commenters) > 0:
        G.add_edge(video1, video2, weight=len(common_commenters))
for n in G.nodes:
    G.nodes[n]['channel'] = Channel_of[n]
    G.nodes[n]['date'] = Date_of_video[n]

len(G.nodes), len(G.edges)
nx.write_gexf(G,"cocommentatori_1.gexf")

In [71]:
barabasi_albert_graph = nx.barabasi_albert_graph(G.order(), 2)

In [ ]:
G = nx.read_gexf("cocommentatori.gexf")

In [ ]:
pagerank_centrality = nx.pagerank(G, alpha=0.85)
top_10_pagerank = sorted(pagerank_centrality.items(), key=lambda x: x[1], reverse=True)[:20]
print("Top 20 by PageRank Centrality:")
for video, centrality in top_10_pagerank:
    # print clickable link
    # get youtube title
    request = youtube.videos().list(
        part="snippet",
        id=video
    )
    print(f"Video: {video}, Centrality: {centrality:.4f}, Channel: {G.nodes[video]['channel']}, Date: {G.nodes[video]['date']}")
    print(request.execute()['items'][0]['snippet']['title'])


In [ ]:
len(G.edges)

In [ ]:
lista_similarity = []
for channel1, channel2 in combinations(channels, 2):
    videos1 = set(Videos_of[channel1])
    videos2 = set(Videos_of[channel2])
    tuples = create_tuples(videos1, videos2)
    similarity = 0
    for video1, video2 in tuples:
        if G.has_node(video1) and G.has_node(video2):
            neighbors1 = set(G.neighbors(video1))
            neighbors2 = set(G.neighbors(video2))
            cosine_similarity = len(neighbors1.intersection(neighbors2)) / math.sqrt(len(neighbors1) * len(neighbors2))
            similarity += cosine_similarity
        similarity += cosine_similarity
    similarity /= len(tuples)
    lista_similarity.append(((channel1, channel2), similarity))
    
    

In [ ]:
for (channel1, channel2), similarity in sorted(lista_similarity, key=lambda x: x[1], reverse=True):
    print(f"Similarità fra {channel1} e {channel2}: {similarity:.4f}")

In [ ]:
G_video = nx.read_gexf("cocommentatori_50_video.gexf")

In [ ]:
eigenvector_centrality = nx.eigenvector_centrality(G_video, max_iter=1000)
top_eigenvector = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:20]
print("Top 20 video by Eigenvector Centrality:")
for video, centrality in top_eigenvector:
    # print clickable link
    # get youtube title
    request = youtube.videos().list(
        part="snippet",
        id=video
    )
    print(f"Video: {video}, Centrality: {centrality:.4f}, Channel: {G_video.nodes[video]['channel']}, Date: {G_video.nodes[video]['date']}")
    print(request.execute()['items'][0]['snippet']['title'])

In [ ]:
media_pesi = 0
media_pesi_stesso_canale = 0
counter = 0

for channel in channels:
    media_pesi = 0
    media_pesi_stesso_canale = 0
    counter = 0
    for edge in G.edges(data=True):
        sorgente = edge[0]
        destinazione = edge[1]
        if G.nodes[sorgente]['channel'] == channel and G.nodes[destinazione]['channel'] == channel:
            media_pesi_stesso_canale += edge[2]['weight']
            counter += 1
        elif (G.nodes[edge[0]]['channel'] == channel or G.nodes[edge[1]]['channel'] == channel) and not (G.nodes[edge[0]]['channel'] == channel and G.nodes[edge[1]]['channel'] == channel):
            media_pesi += edge[2]['weight']
    if counter > 0:
        media_pesi_stesso_canale /= counter
    else:
        media_pesi_stesso_canale = 0
    # count edges involving this channel but not both nodes in the same channel
    total_other_edges = sum(
        1 for edge in G.edges(data=True)
        if (G.nodes[edge[0]]['channel'] == channel or G.nodes[edge[1]]['channel'] == channel)
        and not (G.nodes[edge[0]]['channel'] == channel and G.nodes[edge[1]]['channel'] == channel)
    )
    if total_other_edges > 0:
        media_pesi /= total_other_edges
    else:
        media_pesi = 0
    print(f"Canale: {channel}")
    print("  Media pesi con gli altri canali:", media_pesi)
    print("  Media pesi stesso canale:", media_pesi_stesso_canale)

Canale: Romeo Agresti
  Media pesi (tra canali diversi): 3.584073045402851
  Media pesi stesso canale: 13.879203203969169
Canale: Il BiancoNero
  Media pesi (tra canali diversi): 1.7655160620031731
  Media pesi stesso canale: 1.3635259362913728
Canale: Colpo Gobbo
  Media pesi (tra canali diversi): 3.816249423624251
  Media pesi stesso canale: 10.774487264256814
Canale: Luca Toselli
  Media pesi (tra canali diversi): 3.340151095441954
  Media pesi stesso canale: 9.10530516431925
Canale: lAngolodiKinoshi
  Media pesi (tra canali diversi): 2.9199011330093114
  Media pesi stesso canale: 5.892640619737286


In [ ]:
# weight distribution
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

def plot_weight_distribution(G):
    weights = [data['weight'] for _, _, data in G.edges(data=True)]
    plt.figure(figsize=(10, 6))
    plt.plot(weights, 'o', markersize=2, alpha=0.5)
    plt.title('Weight Distribution of Edges')
    plt.xlabel('Weight')
    plt.ylabel('Frequency')
    # log scale
    plt.xscale('log')
    plt.yscale('log')
    plt.show()
plot_weight_distribution(G)

In [ ]:
print(len(list(nx.local_bridges(G)))>0)

In [ ]:
import seaborn as sns
import scipy
degree=dict(nx.degree(G))
degree_centrality = {n: round(c/(G.order()-1),4) for n,c in degree.items()}
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality=nx.betweenness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G,200)
all_centr = [degree_centrality, closeness_centrality, betweenness_centrality, eigenvector_centrality]
nodes = list(G.nodes())

corrs = [
    [
        scipy.stats.pearsonr(
            [c1[n] for n in nodes],
            [c2[n] for n in nodes]
        )[0]
        for c1 in all_centr
    ]
    for c2 in all_centr
]

centralities = ["degree", "closeness", "betweenness", "eigenvector"]
len(betweenness_centrality)

In [ ]:
total_videos = set(total_videos)

In [ ]:
# degree distribution
def plot_degree_distribution(G):
    degrees = [degree for node, degree in G.degree()]
    unique_degrees = sorted(set(degrees))
    plt.figure(figsize=(10, 6))
    plt.hist(degrees, bins=len(unique_degrees), color='blue', alpha=0.7, edgecolor='black')
    plt.title('Degree Distribution')
    plt.xlabel('Degree')
    plt.ylabel('Frequency')
    plt.show()


In [ ]:
Videos_commented_by["@RomeoAgresti"]

In [ ]:
plt.figure(figsize = (10,5))
sns.heatmap(pd.DataFrame(corrs),annot = True,cmap = 'coolwarm')
plt.xticks([x+0.5 for x in range(4)],centralities, rotation = 0)
plt.yticks([x+0.5 for x in range(4)],centralities, rotation = 0)
plt.title('Pearson correlation\n', weight = 'bold')
plt.show()

In [ ]:
G_utenti = nx.Graph()
utenti = Videos_commented_by.keys()
for user1, user2 in combinations(utenti, 2):
    videos1 = Videos_commented_by[user1]
    videos2 = Videos_commented_by[user2]
    common_videos = videos1.intersection(videos2)
    common_videos = common_videos.intersection(total_videos)
    if len(common_videos) > 0:
        G_utenti.add_edge(user1, user2, weight=len(common_videos))
for n in G_utenti.nodes:
    videos = Videos_commented_by[n]
    if len(videos) == 1:
        G_utenti.nodes[n]['video'] = videos.pop()
       
nx.write_gexf(G_utenti,"co-commentatori_utenti_piu_grande_con_info.gexf")

In [ ]:
G_utenti = nx.read_gexf("co-commentatori_utenti_piu_grande_con_info.gexf")

In [ ]:
for utenti in Commenters_of_channel["Romeo Agresti"]:
    if len(utenti) <2:
        print(utenti)

In [ ]:
similarita = []
for channel1, channel2 in combinations(channels, 2):
    print(channel1, channel2)
    commenters1 = Commenters_of_channel[channel1]
    commenters2 = Commenters_of_channel[channel2]
    similarity = 0
    tuples = create_tuples(commenters1, commenters2, skip_self=False)
    for c1, c2 in tuples:
        if G_utenti.has_node(c1) and G_utenti.has_node(c2):
            neighbors1 = set(G_utenti.neighbors(c1))
            neighbors2 = set(G_utenti.neighbors(c2))
            cosine_similarity = len(neighbors1.intersection(neighbors2)) / math.sqrt(len(neighbors1) * len(neighbors2))
            similarity += cosine_similarity
    similarity /= len(tuples)
    print(f"Similarità fra {channel1} e {channel2}: {similarity}")
    similarita.append(((channel1,channel2),similarity))

In [ ]:
for (channel1, channel2), similarity in sorted(similarita, key=lambda x: x[1], reverse=True):
    print(f"Similarità fra {channel1} e {channel2}: {similarity:.4f}")

<img src="../esercizi_classe/Grafo_Canali.png" width="500">

In [ ]:
G_utenti = nx.read_gexf("co-commentatori_utenti_piu_grande.gexf")

In [ ]:
# find modularity classes
from networkx.algorithms import community
def get_modularity_classes(G):
    partition = community.greedy_modularity_communities(G)
    return partition
classi = get_modularity_classes(G_utenti)
len(classi)

In [ ]:
average_degree = sum(dict(G_utenti.degree()).values()) / len(G_utenti.nodes())
print("Average degree of the user graph:", average_degree)

In [ ]:
eigenvector_centrality = nx.eigenvector_centrality(G_utenti,200)
sorted_eigenvector = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)
print("Top 10 users by eigenvector centrality:")
for user, centrality in sorted_eigenvector[:10]:
    print(f"User: {user}, Eigenvector Centrality: {centrality:.4f}")

In [ ]:
len(G_utenti.nodes), len(G_utenti.edges) 
# (37916, 23021975) len > 0
# (21805, 5129502) len > 1
# (5541, 544245) len > 1 e intersezione con 50 video per canale
# (11873, 4373890) len > 0 e intersezione con 50 video per canale

In [ ]:
# graph with the same degree distribution
def generate_graph_with_same_degree_distribution(G, num_nodes):
    # Create a random graph with the same degree distribution
    degree_sequence = [d for n, d in G.degree()]
    G_random = nx.configuration_model(degree_sequence)
    G_random = nx.Graph(G_random)  # Convert to simple graph
    return G_random
generated_graph = generate_graph_with_same_degree_distribution(G_utenti, len(G_utenti.nodes()))
eigenvector_centrality_2 = nx.eigenvector_centrality(generated_graph,200)
sorted_eigenvector_2 = sorted(eigenvector_centrality_2.items(), key=lambda x: x[1], reverse=True)
print("Top 10 users by eigenvector centrality in generated graph:")


In [ ]:
plot_degree_distribution(G_utenti)
plot_degree_distribution(generated_graph)

In [ ]:
Motta_comments = []
for channel in channels:
    with open(f"comments_{channel}.csv", "r", encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if "motta" in row["Content"].lower() or "tiago" in row["Content"].lower() or "thiago" in row["Content"].lower() :
                Motta_comments.append(row)
            if ("motta" in row["Content"].lower() or "tiago" in row["Content"].lower() or "thiago" in row["Content"].lower() )and "interista" in row["Content"].lower() :
                print(row["Content"])

In [ ]:
Motta_comments

In [21]:
results_dict = {}
with open("colonne.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    headers = next(reader)
    for row in reader:
        channel = row["0"]
        modularity_class = row["modularity_class"]
        if channel not in results_dict:
            results_dict[channel] = {}
        if modularity_class not in results_dict[channel]:
            results_dict[channel][modularity_class] = 0
        results_dict[channel][modularity_class] += 1
        
percentage_dict = {}
for channel, classes in results_dict.items():
    total = sum(classes.values())
    percentage_dict[channel] = {k: v / total * 100 for k, v in classes.items()}

        
    

In [20]:
results_dict

{'Romeo Agresti': {'0': 569, '1': 1},
 'Il BiancoNero': {'0': 563, '1': 180, '2': 68, '3': 54, '4': 4},
 'Colpo Gobbo': {'1': 589, '2': 2, '3': 4, '0': 1},
 'Luca Toselli': {'2': 494, '0': 1, '3': 1, '1': 1},
 'lAngolodiKinoshi': {'3': 630, '1': 2, '2': 1}}

In [22]:
percentage_dict

{'Romeo Agresti': {'0': 99.82456140350877, '1': 0.17543859649122806},
 'Il BiancoNero': {'0': 64.78711162255466,
  '1': 20.713463751438436,
  '2': 7.825086306098965,
  '3': 6.214039125431531,
  '4': 0.46029919447640966},
 'Colpo Gobbo': {'1': 98.8255033557047,
  '2': 0.33557046979865773,
  '3': 0.6711409395973155,
  '0': 0.16778523489932887},
 'Luca Toselli': {'2': 99.3963782696177,
  '0': 0.2012072434607646,
  '3': 0.2012072434607646,
  '1': 0.2012072434607646},
 'lAngolodiKinoshi': {'3': 99.52606635071089,
  '1': 0.315955766192733,
  '2': 0.1579778830963665}}

In [18]:
dictionary = {
    "0": "Romeo Agresti",
    "1": "Colpo Gobbo",
    "2": "Luca Toselli",
    "3": "lAngolodiKinoshi",
    "4": "Il BiancoNero",
}

with open("colonne.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    headers = next(reader)
    for row in reader:
        if row["0"] == "Il BiancoNero" and row["modularity_class"] != "0":
            # print youtube link
            video_id = row["Id"]
            print(dictionary[row["modularity_class"]])
            print(f"https://www.youtube.com/watch?v={video_id}")
            

Colpo Gobbo
https://www.youtube.com/watch?v=XHHGRjdrF2k
Colpo Gobbo
https://www.youtube.com/watch?v=c71kGYdIY-4
Colpo Gobbo
https://www.youtube.com/watch?v=C6hGe3HExb4
Colpo Gobbo
https://www.youtube.com/watch?v=7pZSDj9dRfM
Colpo Gobbo
https://www.youtube.com/watch?v=JC4W5VDPefA
Colpo Gobbo
https://www.youtube.com/watch?v=5dKixJ-1XPA
Luca Toselli
https://www.youtube.com/watch?v=XIF9C2T00wc
Colpo Gobbo
https://www.youtube.com/watch?v=R22psUh15Jo
Colpo Gobbo
https://www.youtube.com/watch?v=IL3-4cRNbu4
Colpo Gobbo
https://www.youtube.com/watch?v=YCh9X4ArBkw
Colpo Gobbo
https://www.youtube.com/watch?v=bsFunlwppZg
Colpo Gobbo
https://www.youtube.com/watch?v=XeUyaEnUjdY
lAngolodiKinoshi
https://www.youtube.com/watch?v=QPPQzBmkV7g
lAngolodiKinoshi
https://www.youtube.com/watch?v=roY1UO3_oa0
lAngolodiKinoshi
https://www.youtube.com/watch?v=iFo-w65Pryw
Colpo Gobbo
https://www.youtube.com/watch?v=xYwPY2mrBCI
Colpo Gobbo
https://www.youtube.com/watch?v=tt06F7D5HmY
Colpo Gobbo
https://www.youtube.

In [25]:
with open("colonne.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    headers = next(reader)
    for row in reader:
        if row["modularity_class"] == "4":
            print(f"https://www.youtube.com/watch?v={row['Id']}")


https://www.youtube.com/watch?v=Z1FUEPag9pY
https://www.youtube.com/watch?v=UY4tOKr4Koo
https://www.youtube.com/watch?v=IUKqLfvz6hk
https://www.youtube.com/watch?v=-2meiHifoNY
